## CSCI 5654 - Linear Programming - Project

***
#### Team Members
#### 1. Ketan Ramesh
#### 2. Shreyas Gopalakrishna
***

## Vehicle Routing Problem with Simultaneous Pickup and Delivery

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import random
import pulp
import gurobipy as gp
from gurobipy import GRB

## Vehicle Routing with Simultaneous Pickup and Delivery 
#### Link for book: 
#### The below code solves the vehicle routing problem with Simultaneous Pickup and Delivery by following the methodology mentioned in the above book.
#### The formulation is as follows:

## Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSDP) - Model 1
​
Paper Link - [Vehicle Routing Problem with Deliveries and Pickups: Modelling Issues and Meta-heuristics Solution Approaches](https://core.ac.uk/download/pdf/19477982.pdf)
​
<div align="justify">
The code provided below creates a <code>class</code> to frame and solve the Vehicle Routing Problem with Simultaneous Pickup and Delivery (VRPSDP) using the two-index flow formulation as mentioned in the paper. The formulation of the problem as a Mixed ILP is as follows:
</div>
​
$$
min \sum_{i=0}^{n} \sum_{j=0}^{n} d_{ij} x_{ij}
$$
​
$$
s.t. \sum_{i=0}^{n} x_{ij} = 1, j \in \{1, ..., n\}, - (3.1)
$$
​
$$
\sum_{i=0}^{n} x_{ji} = 1, j \in \{1, ..., n\}, - (3.2)
$$
​
$$
\sum_{i=0}^{n} R_{ij} - q_{j} = \sum_{i=0}^{n} R_{ji}, j \in \{1, ..., n\}, - (3.3)
$$
​
$$
\sum_{i=0}^{n} P_{ij} + b_{j} = \sum_{i=0}^{n} P_{ji}, j \in \{1, ..., n\}, - (3.4)
$$
​
$$
\sum_{i=1}^{n} R_{i0} = 0, - (3.5)
$$
​
$$
\sum_{i=1}^{n} P_{0i} = 0, - (3.6)
$$
​
$$
R_{ij} + P_{ij} \leq C x_{ij}, i, j \in \{0, ..., n\}, - (3.7)
$$
​
$$
x_{ij} = \{0, 1\}, i, j \in \{1, ... n\}
$$
​
$$
R_{ij}, P_{ij} \geq 0, i, j \in \{1, ..., n\}
$$
​
where,<br />
​
d<sub>ij</sub> = Cost to travel from node i to j.<br />
q<sub>i</sub> = The delivery demand of node i.<br />
b<sub>i</sub> = The pickup demand of node i.<br />
C = Vehicle capacity.<br />
​
<b>Decision Variables:</b>
1. x<sub>ij</sub> = {<b>1</b>, if edge (i, j) present in any route. <b>0</b>, otherwise}.
2. R<sub>ij</sub> = The amount of delivery goods on board on arc ij.
3. P<sub>ij</sub> = The amount of pickup goods on board on arc ij.
​
<div align="justify">
The objective is to minimize the sum of the distances of all routes that satisfy the given constraints. The constraints (3.1) and (3.2) ensure that every node is visited exactly once (every node leads to only one other node, every node must be visited by only one other node). The constraints (3.3) and (3.4) ensure that the flow conservation is met (at a node <i>j</i>, the amount of delivery load after servicing delivery of node <i>j</i>, must be equal to pickup load at the same node. The pickup load condition is similar in nature). The constraints (3.5) and (3.6) ensure that the depot has zero pickup and delivery load. The constraint (3.7) ensures that the pickup and demand loads do not exceed the capacity of the vehicle at all nodes in the routes.  
</div>
​

In [17]:
class CVRP_SPD_GUROBI:
	def __init__(self, numberOfCustomers, numberOfVehicles, capacityOfVehicle, pickups, delivery, costMatrix):
		self.numberOfCustomers = numberOfCustomers
		self.numberOfVehicles  = numberOfVehicles
		self.capacityOfVehicle = capacityOfVehicle
		self.pickups = pickups
		self.delivery = delivery
		self.costMatrix        = costMatrix
		self.initializeLP()
	
	def initializeLP(self):
		self.cvrpLP = gp.Model("CVRP_PD")
		objective = None
		x, y, z = [], [], []
		D, d = [], []
		P, p = [], []
		
		K = self.numberOfVehicles
		N = len(costMatrix)

		# create variables
		for i in range(N):
			xTemp1 = []
			for j in range(N):
				xTemp2 = []
				for k in range(K):
					xTemp3 = self.cvrpLP.addVar(name='x('+str(i)+','+str(j)+','+str(k)+')', vtype=GRB.BINARY, lb=0, ub=1)
					xTemp2.append(xTemp3)
				xTemp1.append(xTemp2)
			x.append(xTemp1)
		print(type(x[0][0][0]))
		
		for i in range(N):
			yTemp1 = []
			for j in range(N):
				yTemp2 = self.cvrpLP.addVar(name='y('+str(i)+','+str(j)+')', vtype=GRB.CONTINUOUS, lb=0)
				yTemp1.append(yTemp2)
			y.append(yTemp1)
		
		for i in range(N):
			zTemp1 = []
			for j in range(N):
				zTemp2 = self.cvrpLP.addVar(name='z('+str(i)+','+str(j)+')', vtype=GRB.CONTINUOUS, lb=0)
				zTemp1.append(zTemp2)
			z.append(zTemp1)
		
		# objective function
		for i in range(N):
			for j in range(N):
				for k in range(K):
					objective += self.costMatrix[i][j] * x[i][j][k]
		self.cvrpLP.setObjective(objective, GRB.MINIMIZE)

		#constraints

		#constraint-1
		for i in range(1, N):
			constraint1 = None
			for j in range(N):
				for k in range(K):
					if(constraint1 == None):
						constraint1 = x[i][j][k]
					else:
						constraint1 = constraint1 + x[i][j][k]
			self.cvrpLP.addConstr(constraint1 == 1)
		
		#constraint-2
		for k in range(K):
			for i in range(N):
				constraint2a, constraint2b = None, None
				for j in range(N):
					if(constraint2a == None):
						constraint2a = x[i][j][k]
					else:
						constraint2a = constraint2a + x[i][j][k]
					if(constraint2b == None):
						constraint2b = x[i][j][k]
					else:
						constraint2b = constraint2b + x[i][j][k]
					# constraint2a += x[i][j][k]
					# constraint2b += x[j][i][k]
				self.cvrpLP.addConstr(constraint2a - constraint2b == 0)
		
		#constraint-3
		for k in range(K):
			constraint3 = None
			for i in range(N):
				if(constraint3 == None):
						constraint3 = x[0][i][k]
				else:
					constraint3 = constraint3 + x[0][i][k]
				# constraint3 += x[0][i][k]
			self.cvrpLP.addConstr(constraint3 <= 1)
		
		# constraint-4
		for i in range(N):
			for j in range(N):
				constraint4a = None
				for k in range(K):
					if(constraint4a == None):
						constraint4a = x[i][j][k]
					else:
						constraint4a = constraint4a + x[i][j][k]
				self.cvrpLP.addConstr(y[i][j] + z[i][j] <= self.capacityOfVehicle * constraint4a)
		
		#constraint-5
		for i in range(1, N):
			constraint5a, constraint5b = None, None
			for j in range(N):
				if(constraint5a == None):
					constraint5a = y[i][j]
				else:
					constraint5a = constraint5a + y[i][j]
				if(constraint5b == None):
					constraint5b = y[j][i]
				else:
					constraint5b = constraint5b + y[j][i]
			self.cvrpLP.addConstr(constraint5a - constraint5b == self.pickups[i])

		#constraint-6
		for i in range(1, N):
			constraint6a, constraint6b = None, None
			for j in range(N):
				if(constraint6a == None):
					constraint6a = z[j][i]
				else:
					constraint6a = constraint6a + z[j][i]
				if(constraint6b == None):
					constraint6b = z[i][j]
				else:
					constraint6b = constraint6b + z[i][j]
			self.cvrpLP.addConstr(constraint6a - constraint6b == self.delivery[i])
		
		print(self.cvrpLP)

	def solve(self):
		status = self.cvrpLP.optimize()#pulp.solvers.PULP_CBC_CMD(self.cvrpLP))
		print(status)
	
	def getResult(self):
		print("Objective value: ", self.cvrpLP.ObjVal)
		for v in self.cvrpLP.getVars():
			print(v.varName, " = ", v.x)


### TESTING PRE-DEFINED DATASET - N=13

In [19]:
costMatrix = [[0,9,14,23,32,50,21,49,30,27,35,28,18],
[9,0,21,22,36,52,24,51,36,37,41,30,20],
[14,21,0,25,38,5,31,7,36,43,29,7,6],
[23,22,25,0,42,12,35,17,44,31,31,11,6],
[32,36,38,42,0,22,37,16,46,37,29,13,14],
[50,52,5,12,22,0,41,23,10,39,9,17,16],
[21,24,31,35,37,41,0,26,21,19,10,25,12],
[49,51,7,17,16,23,26,0,30,28,16,27,12],
[30,36,36,44,46,10,21,30,0,25,22,10,20],
[27,37,43,31,37,39,19,28,25,0,20,16,8],
[35,41,29,31,29,9,10,16,22,20,0,10,10],
[28,30,7,11,13,17,25,27,10,16,10,0,10],
[18,20, 6, 6,14,16,12,12,20,8, 10,10,0]]

delivery = [0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700, 1100]
pickups = [1100, 0, 1200, 1700, 1500, 1400, 1700, 1400, 1200, 1900, 1800, 1600, 1700]
capacityOfVehicle = 6000
numberOfVehicles = 4
numberOfCustomers = len(costMatrix) - 1

In [20]:
lp = CVRP_SPD_GUROBI(numberOfCustomers, numberOfVehicles, capacityOfVehicle, pickups, delivery, costMatrix)
lp.solve()
lp.getResult()

<class 'gurobipy.Var'>
<gurobi.Model Continuous instance CVRP_PD: 0 constrs, 0 vars, No parameter changes>
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 261 rows, 1014 columns and 2266 nonzeros
Model fingerprint: 0x5dd397a6
Variable types: 338 continuous, 676 integer (676 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [5e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 76 rows and 521 columns
Presolve time: 0.01s
Presolved: 185 rows, 493 columns, 1250 nonzeros
Variable types: 312 continuous, 181 integer (181 binary)

Root relaxation: objective 1.889167e+02, 284 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  188.91667    0   19          -  188.91667      -     -    0s
     0     0  212.04887    0   30          -  212.04887      - 

x(9,11,0)  =  0.0
x(9,11,1)  =  0.0
x(9,11,2)  =  0.0
x(9,11,3)  =  -0.0
x(9,12,0)  =  0.0
x(9,12,1)  =  0.0
x(9,12,2)  =  0.0
x(9,12,3)  =  -0.0
x(10,0,0)  =  0.0
x(10,0,1)  =  0.0
x(10,0,2)  =  0.0
x(10,0,3)  =  0.0
x(10,1,0)  =  0.0
x(10,1,1)  =  0.0
x(10,1,2)  =  0.0
x(10,1,3)  =  -0.0
x(10,2,0)  =  0.0
x(10,2,1)  =  0.0
x(10,2,2)  =  0.0
x(10,2,3)  =  -0.0
x(10,3,0)  =  0.0
x(10,3,1)  =  0.0
x(10,3,2)  =  0.0
x(10,3,3)  =  -0.0
x(10,4,0)  =  0.0
x(10,4,1)  =  0.0
x(10,4,2)  =  0.0
x(10,4,3)  =  -0.0
x(10,5,0)  =  0.0
x(10,5,1)  =  0.0
x(10,5,2)  =  0.0
x(10,5,3)  =  -0.0
x(10,6,0)  =  0.0
x(10,6,1)  =  0.0
x(10,6,2)  =  0.0
x(10,6,3)  =  -0.0
x(10,7,0)  =  0.0
x(10,7,1)  =  0.0
x(10,7,2)  =  0.0
x(10,7,3)  =  0.0
x(10,8,0)  =  0.0
x(10,8,1)  =  0.0
x(10,8,2)  =  0.0
x(10,8,3)  =  -0.0
x(10,9,0)  =  0.0
x(10,9,1)  =  0.0
x(10,9,2)  =  0.0
x(10,9,3)  =  1.0
x(10,10,0)  =  0.0
x(10,10,1)  =  0.0
x(10,10,2)  =  0.0
x(10,10,3)  =  0.0
x(10,11,0)  =  0.0
x(10,11,1)  =  0.0
x(10,11,2)  =

z(12,10)  =  0.0
z(12,11)  =  0.0
z(12,12)  =  0.0


### TESTING PRE-DEFINED DATASET - E-n22-k4.vrp = CE22P

In [23]:
# E-n22-k4.vrp = CE22P
xCoordinates = [145, 151, 159, 130, 128, 163, 146, 161, 142, 163, 148, 128, 156, 129, 146, 164, 141, 147, 164, 129, 155, 139]
yCoordinates = [215, 164, 261, 254, 252, 247, 246, 242, 239, 236, 232, 231, 217, 214, 208, 208, 206, 193, 193, 189, 185, 182]
costMatrix = np.ndarray(shape=(len(xCoordinates), len(yCoordinates)))
for i in range(len(xCoordinates)):
    for j in range(len(yCoordinates)):
        costMatrix[i][j] = float(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
delivery = [0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500, 700]
pickups = [700, 0, 1100, 700, 800, 1400, 2100, 400, 800, 100, 500, 600, 1200, 1300, 1300, 300, 900, 2100, 1000, 900, 2100, 1000, 900, 2500]
capacityOfVehicle = 6000
numberOfVehicles = 5

In [24]:
lp = CVRP_SPD_GUROBI(numberOfCustomers, numberOfVehicles, capacityOfVehicle, pickups, delivery, costMatrix)
lp.solve()
lp.getResult()

<class 'gurobipy.Var'>
<gurobi.Model Continuous instance CVRP_PD: 0 constrs, 0 vars, No parameter changes>
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 662 rows, 3388 columns and 7572 nonzeros
Model fingerprint: 0x8baebb9f
Variable types: 968 continuous, 2420 integer (2420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [3e+00, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+03]
Presolve removed 152 rows and 1938 columns
Presolve time: 0.02s
Presolved: 510 rows, 1450 columns, 3740 nonzeros
Variable types: 924 continuous, 526 integer (526 binary)

Root relaxation: objective 2.869331e+02, 1016 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  286.93314    0   42          -  286.93314      -     -    0s
     0     0  306.59155    0   56          -  306.59155  

x(3,19,1)  =  0.0
x(3,19,2)  =  0.0
x(3,19,3)  =  0.0
x(3,19,4)  =  0.0
x(3,20,0)  =  0.0
x(3,20,1)  =  0.0
x(3,20,2)  =  0.0
x(3,20,3)  =  0.0
x(3,20,4)  =  0.0
x(3,21,0)  =  0.0
x(3,21,1)  =  0.0
x(3,21,2)  =  0.0
x(3,21,3)  =  0.0
x(3,21,4)  =  0.0
x(4,0,0)  =  0.0
x(4,0,1)  =  0.0
x(4,0,2)  =  0.0
x(4,0,3)  =  0.0
x(4,0,4)  =  0.0
x(4,1,0)  =  0.0
x(4,1,1)  =  0.0
x(4,1,2)  =  0.0
x(4,1,3)  =  0.0
x(4,1,4)  =  0.0
x(4,2,0)  =  0.0
x(4,2,1)  =  0.0
x(4,2,2)  =  0.0
x(4,2,3)  =  0.0
x(4,2,4)  =  0.0
x(4,3,0)  =  0.0
x(4,3,1)  =  0.0
x(4,3,2)  =  0.0
x(4,3,3)  =  0.0
x(4,3,4)  =  1.0
x(4,4,0)  =  0.0
x(4,4,1)  =  0.0
x(4,4,2)  =  0.0
x(4,4,3)  =  0.0
x(4,4,4)  =  0.0
x(4,5,0)  =  0.0
x(4,5,1)  =  0.0
x(4,5,2)  =  0.0
x(4,5,3)  =  0.0
x(4,5,4)  =  0.0
x(4,6,0)  =  0.0
x(4,6,1)  =  0.0
x(4,6,2)  =  0.0
x(4,6,3)  =  0.0
x(4,6,4)  =  0.0
x(4,7,0)  =  0.0
x(4,7,1)  =  0.0
x(4,7,2)  =  0.0
x(4,7,3)  =  0.0
x(4,7,4)  =  0.0
x(4,8,0)  =  0.0
x(4,8,1)  =  0.0
x(4,8,2)  =  0.0
x(4,8,3)  =  0.0


x(9,0,4)  =  0.0
x(9,1,0)  =  0.0
x(9,1,1)  =  0.0
x(9,1,2)  =  0.0
x(9,1,3)  =  0.0
x(9,1,4)  =  0.0
x(9,2,0)  =  0.0
x(9,2,1)  =  0.0
x(9,2,2)  =  0.0
x(9,2,3)  =  0.0
x(9,2,4)  =  0.0
x(9,3,0)  =  0.0
x(9,3,1)  =  0.0
x(9,3,2)  =  0.0
x(9,3,3)  =  0.0
x(9,3,4)  =  0.0
x(9,4,0)  =  0.0
x(9,4,1)  =  0.0
x(9,4,2)  =  0.0
x(9,4,3)  =  0.0
x(9,4,4)  =  0.0
x(9,5,0)  =  0.0
x(9,5,1)  =  0.0
x(9,5,2)  =  0.0
x(9,5,3)  =  0.0
x(9,5,4)  =  0.0
x(9,6,0)  =  0.0
x(9,6,1)  =  0.0
x(9,6,2)  =  0.0
x(9,6,3)  =  0.0
x(9,6,4)  =  0.0
x(9,7,0)  =  0.0
x(9,7,1)  =  0.0
x(9,7,2)  =  0.0
x(9,7,3)  =  0.0
x(9,7,4)  =  1.0
x(9,8,0)  =  0.0
x(9,8,1)  =  0.0
x(9,8,2)  =  0.0
x(9,8,3)  =  0.0
x(9,8,4)  =  0.0
x(9,9,0)  =  0.0
x(9,9,1)  =  0.0
x(9,9,2)  =  0.0
x(9,9,3)  =  0.0
x(9,9,4)  =  0.0
x(9,10,0)  =  0.0
x(9,10,1)  =  0.0
x(9,10,2)  =  0.0
x(9,10,3)  =  0.0
x(9,10,4)  =  0.0
x(9,11,0)  =  0.0
x(9,11,1)  =  0.0
x(9,11,2)  =  0.0
x(9,11,3)  =  0.0
x(9,11,4)  =  0.0
x(9,12,0)  =  0.0
x(9,12,1)  =  0.0
x(

x(14,10,2)  =  0.0
x(14,10,3)  =  0.0
x(14,10,4)  =  0.0
x(14,11,0)  =  0.0
x(14,11,1)  =  0.0
x(14,11,2)  =  0.0
x(14,11,3)  =  0.0
x(14,11,4)  =  0.0
x(14,12,0)  =  0.0
x(14,12,1)  =  0.0
x(14,12,2)  =  0.0
x(14,12,3)  =  0.0
x(14,12,4)  =  0.0
x(14,13,0)  =  0.0
x(14,13,1)  =  0.0
x(14,13,2)  =  0.0
x(14,13,3)  =  0.0
x(14,13,4)  =  0.0
x(14,14,0)  =  0.0
x(14,14,1)  =  0.0
x(14,14,2)  =  0.0
x(14,14,3)  =  0.0
x(14,14,4)  =  0.0
x(14,15,0)  =  0.0
x(14,15,1)  =  0.0
x(14,15,2)  =  0.0
x(14,15,3)  =  0.0
x(14,15,4)  =  0.0
x(14,16,0)  =  0.0
x(14,16,1)  =  0.0
x(14,16,2)  =  0.0
x(14,16,3)  =  0.0
x(14,16,4)  =  0.0
x(14,17,0)  =  0.0
x(14,17,1)  =  0.0
x(14,17,2)  =  0.0
x(14,17,3)  =  0.0
x(14,17,4)  =  0.0
x(14,18,0)  =  0.0
x(14,18,1)  =  0.0
x(14,18,2)  =  0.0
x(14,18,3)  =  0.0
x(14,18,4)  =  0.0
x(14,19,0)  =  0.0
x(14,19,1)  =  0.0
x(14,19,2)  =  0.0
x(14,19,3)  =  0.0
x(14,19,4)  =  0.0
x(14,20,0)  =  0.0
x(14,20,1)  =  0.0
x(14,20,2)  =  0.0
x(14,20,3)  =  0.0
x(14,20,4)  

x(19,8,2)  =  0.0
x(19,8,3)  =  0.0
x(19,8,4)  =  0.0
x(19,9,0)  =  0.0
x(19,9,1)  =  0.0
x(19,9,2)  =  0.0
x(19,9,3)  =  0.0
x(19,9,4)  =  0.0
x(19,10,0)  =  0.0
x(19,10,1)  =  0.0
x(19,10,2)  =  0.0
x(19,10,3)  =  0.0
x(19,10,4)  =  0.0
x(19,11,0)  =  0.0
x(19,11,1)  =  0.0
x(19,11,2)  =  0.0
x(19,11,3)  =  0.0
x(19,11,4)  =  0.0
x(19,12,0)  =  0.0
x(19,12,1)  =  0.0
x(19,12,2)  =  0.0
x(19,12,3)  =  0.0
x(19,12,4)  =  0.0
x(19,13,0)  =  0.0
x(19,13,1)  =  0.0
x(19,13,2)  =  0.0
x(19,13,3)  =  0.0
x(19,13,4)  =  0.0
x(19,14,0)  =  0.0
x(19,14,1)  =  0.0
x(19,14,2)  =  0.0
x(19,14,3)  =  0.0
x(19,14,4)  =  0.0
x(19,15,0)  =  0.0
x(19,15,1)  =  0.0
x(19,15,2)  =  0.0
x(19,15,3)  =  0.0
x(19,15,4)  =  0.0
x(19,16,0)  =  0.0
x(19,16,1)  =  0.0
x(19,16,2)  =  0.0
x(19,16,3)  =  0.0
x(19,16,4)  =  0.0
x(19,17,0)  =  0.0
x(19,17,1)  =  0.0
x(19,17,2)  =  0.0
x(19,17,3)  =  0.0
x(19,17,4)  =  0.0
x(19,18,0)  =  0.0
x(19,18,1)  =  0.0
x(19,18,2)  =  0.0
x(19,18,3)  =  0.0
x(19,18,4)  =  0.0
x

y(15,8)  =  0.0
y(15,9)  =  0.0
y(15,10)  =  0.0
y(15,11)  =  0.0
y(15,12)  =  0.0
y(15,13)  =  0.0
y(15,14)  =  0.0
y(15,15)  =  0.0
y(15,16)  =  0.0
y(15,17)  =  1500.0
y(15,18)  =  0.0
y(15,19)  =  0.0
y(15,20)  =  0.0
y(15,21)  =  0.0
y(16,0)  =  0.0
y(16,1)  =  0.0
y(16,2)  =  0.0
y(16,3)  =  0.0
y(16,4)  =  0.0
y(16,5)  =  0.0
y(16,6)  =  0.0
y(16,7)  =  0.0
y(16,8)  =  0.0
y(16,9)  =  0.0
y(16,10)  =  0.0
y(16,11)  =  0.0
y(16,12)  =  0.0
y(16,13)  =  0.0
y(16,14)  =  4500.0
y(16,15)  =  0.0
y(16,16)  =  0.0
y(16,17)  =  0.0
y(16,18)  =  0.0
y(16,19)  =  0.0
y(16,20)  =  0.0
y(16,21)  =  0.0
y(17,0)  =  0.0
y(17,1)  =  0.0
y(17,2)  =  0.0
y(17,3)  =  0.0
y(17,4)  =  0.0
y(17,5)  =  0.0
y(17,6)  =  0.0
y(17,7)  =  0.0
y(17,8)  =  0.0
y(17,9)  =  0.0
y(17,10)  =  0.0
y(17,11)  =  0.0
y(17,12)  =  0.0
y(17,13)  =  0.0
y(17,14)  =  0.0
y(17,15)  =  0.0
y(17,16)  =  3600.0
y(17,17)  =  0.0
y(17,18)  =  0.0
y(17,19)  =  0.0
y(17,20)  =  0.0
y(17,21)  =  0.0
y(18,0)  =  5000.0
y(18,1) 